In [141]:
import numpy as np
import pandas as pd

In [142]:
import prepostprocessing.cleaning as cleaning
import prepostprocessing.pre_processing as preproc
from qapf import qapf
from qapf import cipw

___

# CIPWFULL normalized data

## Convert to CIPWFULL format

**I used an arbitrary version of the mineralogical data, not sure if this is the correct one** -->is a good one

In [143]:
mineralogy_original = pd.read_excel("../_CIPW/CIPW/AREA6/interpolated_data.xlsx", index_col=0)
# Only iterested in sample number and oxides
mineralogy = mineralogy_original[['SiO2', 'TiO2', 'Al2O3', 'Fe2O3','FeO', 'MnO', 'MgO', 'CaO', 'Na2O', 'K2O', 'P2O5', 'l.i.', 'oth']]
mineralogy.columns

Index(['SiO2', 'TiO2', 'Al2O3', 'Fe2O3', 'FeO', 'MnO', 'MgO', 'CaO', 'Na2O',
       'K2O', 'P2O5', 'l.i.', 'oth'],
      dtype='object')

In [144]:
mineralogy = mineralogy.dropna()

In [145]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [146]:
# Remove 'l.i.'' and 'oth' columns and immediately normalize data
mineralogy = preproc.normalize(mineralogy.drop(["l.i.", "oth"], axis=1))

In [147]:
# Check for closed sum
assert all(np.isclose(mineralogy.sum(axis=1), 100.0))

In [148]:
mineralogy.head()

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5
12278,71.077514,0.331106,15.224189,0.957943,2.311173,0.070808,0.644457,1.537197,3.611444,4.198481,0.035690
12279,71.031588,0.333050,15.240039,0.961267,2.323062,0.071032,0.651569,1.551644,3.611192,4.189488,0.036068
12280,70.983555,0.335083,15.256490,0.964733,2.335490,0.071266,0.659050,1.566815,3.610915,4.180137,0.036466
12281,70.910899,0.338072,15.280819,0.972133,2.354027,0.071605,0.670756,1.589535,3.611193,4.163447,0.037516
12282,70.858722,0.340280,15.298326,0.975870,2.367499,0.071858,0.679003,1.606179,3.610853,4.153453,0.037957


In [149]:
mineralogy

,SiO2,TiO2,Al2O3,Fe2O3,FeO,MnO,MgO,CaO,Na2O,K2O,P2O5
12278,71.077514,0.331106,15.224189,0.957943,2.311173,0.070808,0.644457,1.537197,3.611444,4.198481,0.035690
12279,71.031588,0.333050,15.240039,0.961267,2.323062,0.071032,0.651569,1.551644,3.611192,4.189488,0.036068
12280,70.983555,0.335083,15.256490,0.964733,2.335490,0.071266,0.659050,1.566815,3.610915,4.180137,0.036466
12281,70.910899,0.338072,15.280819,0.972133,2.354027,0.071605,0.670756,1.589535,3.611193,4.163447,0.037516
12282,70.858722,0.340280,15.298326,0.975870,2.367499,0.071858,0.679003,1.606179,3.610853,4.153453,0.037957
...,...,...,...,...,...,...,...,...,...,...,...
209133,73.832904,0.211784,14.016816,0.955374,1.582013,0.055083,0.311414,0.772537,3.683076,4.516002,0.062998
209134,73.790305,0.213404,14.038530,0.959622,1.592653,0.055316,0.315811,0.781886,3.683571,4.505099,0.063801
209160,72.430501,0.265888,14.659350,1.093615,1.932336,0.062455,0.474585,1.107632,3.692482,4.187437,0.093718
209161,72.540911,0.261678,14.614534,1.080643,1.905195,0.061914,0.460255,1.079678,3.691559,4.213674,0.089961


In [151]:
mineralogy1 = mineralogy.iloc[0:45000]
mineralogy2 = mineralogy.iloc[45000:90000]

In [152]:
cipw.convert_to_CIPWFULL_format(mineralogy1, "../_CIPW/CIPW/AREA6/area6_1.txt", dataset_name="Vistelius (1995)")
cipw.convert_to_CIPWFULL_format(mineralogy2, "../_CIPW/CIPW/AREA6/area6_2.txt", dataset_name="Vistelius (1995)")
#cipw.convert_to_CIPWFULL_format(mineralogy3, "../_CIPW/CIPW/AREA6/area6_3.txt", dataset_name="Vistelius (1995)")
#cipw.convert_to_CIPWFULL_format(mineralogy4, "../_CIPW/CIPW/AREA6/area6_4.txt", dataset_name="Vistelius (1995)")

_____

## Extract results from CIPWFULL results file

**I only selected quartz, plagioclase and alkali feldspar here as they are of interest to us for the QAPF classification**  
**Feel free to extract addiotnal minerals if they would be of any interest to you**

**Not sure what to do with albite in the final assignments as by the book it should be accounted for as an alkali feldspar and thus be included in the sum of plagioclase but then the resulting plagioclase contents seem quite low.   
This is something you might look into although the method used here seems to be correct according to te resulting QAPF diagram.**

--> indeed when reading the paper of Streckeisen (and other sources), albite should be accounted for as an alkali feldspar, also saw that the datapoints were generally located more to the right side (plagioclase), maybe an overestimation of plagioclase?

--> I changed the settings below so that albite belongs to the alkali feldspars, Think this figure should be more correct according to the papers, but the cotent of plagioclase is indeed quite low. 

In [153]:
df_final1 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA6/area6_1_results_excel.txt")
df_final2 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA6/area6_2_results_excel.txt")
#df_final3 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA6/area6_3_results_excel.txt")
#df_final4 = cipw.extract_CIPW_results("../_CIPW/CIPW/AREA6/area6_4_results_excel.txt")

In [154]:
df_final = pd.concat([df_final1, df_final2])

In [155]:
df_final

,Q,P,K
SAMPLE,,,
12278,28.463026,34.893675,27.865046
12279,28.405086,34.960922,27.811709
12280,28.344086,35.031558,27.756232
12281,28.259798,35.139463,27.657833
12282,28.192768,35.216547,27.598458
...,...,...,...
209133,31.525938,31.849747,30.067191
209134,31.482496,31.906550,29.996671
209160,29.627684,34.110778,27.720580


## QAPF classification

In [156]:
df_final["QAPF"] = qapf.check_QAPF_region(df_final)

In [157]:
df_final["QAPF"].unique()

array(['monzo granite', 'granodiorite',
       'quartz monzodiorite\nquartz monzogabbro',
       'monzodiorite monzogabbro'], dtype=object)

In [158]:
df_final.to_excel("../_CIPW/CIPW/AREA6/qapf.xlsx", index=True)

-------

In [159]:
df_final.iloc[:, 3].value_counts().to_excel("../_CIPW/CIPW/AREA6/QAPF_counts.xlsx")

-----

In [160]:
QAPF_control = pd.read_excel("../_CIPW/CIPW/1_normalized.xlsx", index_col=0)
Area = pd.read_excel("../_INTERPOLATION/area6.xlsx", index_col=None, usecols = ['area', 'Y_UTM', 'X_UTM', 'ZoneNumber', 'ZoneLetter'])

In [161]:
QAPF_control = pd.concat([Area, QAPF_control], axis = 1)
QAPF_control = QAPF_control.dropna()
QAPF_control

,Y_UTM,X_UTM,ZoneNumber,ZoneLetter,area,Q,P,K,QAPF
1,718647.819565,5.235770e+06,53.0,T,6.0,52.549507,17.912818,22.931202,monzo granite
2,731796.073283,5.259752e+06,53.0,T,6.0,45.940433,19.086384,32.357653,monzo granite
3,755477.629709,5.262227e+06,53.0,T,6.0,50.739986,6.408381,37.371890,syeno granite
4,734741.917204,5.301092e+06,53.0,T,6.0,41.407116,23.232361,27.590113,monzo granite
5,661219.827466,5.226625e+06,53.0,T,6.0,54.633030,17.945563,19.744849,monzo granite
...,...,...,...,...,...,...,...,...,...
146,671281.488145,5.177993e+06,53.0,T,6.0,39.742012,34.500514,19.826222,monzo granite
147,735289.609871,5.294808e+06,53.0,T,6.0,34.818058,32.875242,27.974829,monzo granite
148,670787.218612,5.216880e+06,53.0,T,6.0,39.027565,21.638880,34.288132,monzo granite
149,746609.482852,5.293856e+06,53.0,T,6.0,39.312107,26.125557,29.271148,monzo granite


In [163]:
QAPF_control.to_excel("../_CIPW/CIPW/AREA6/QAPF_control_points.xlsx", index=True)